### Importing Libraries:

In [10]:
# Import necessary libraries
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image
from time import sleep
import os
import numpy as np
from sklearn.metrics import classification_report

from amb_sdk.sdk import DarwinSdk

### Configure Darwin:

In [11]:
# Login
ds = DarwinSdk()
file = open("login.txt", "r")
username = file.readline(0)
password = file.readline(1)
ds.set_url('https://amb-demo-api.sparkcognition.com/v1/')
status, msg = ds.auth_login_user('username', 'password')
if not status:
    print(msg)
    

#### Data Path
Make sure to set this to your local machine's path to the data.

In [ ]:
path = '/Issued_construction_Permits.csv/'

### Importing Data:
Data used in this project:
https://data.austintexas.gov/Building-and-Development/Issued-Construction-Permits/3syk-w9eu

In [6]:
filename = "Issued_Construction_Permits.csv"
data = pd.read_csv(filename, skipinitialspace=True)

data.head()

C:\Users\Neal\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (52,54,56,58,59,60,61,62,63,64,65,66) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Permit Type,Permit Type Desc,Permit Num,Permit Class Mapped,Permit Class,Work Class,Condominium,Project Name,Description,TCAD ID,...,Contractor Zip,Applicant Full Name,Applicant Organization,Applicant Phone,Applicant Address 1,Applicant Address 2,Applicant City,Applicant Zip,Certificate Of Occupancy,Total Lot SQFT
0,EP,Electrical Permit,2007-105766 EP,Residential,R- 434 Addition & Alterations,Addition,No,1407 ALAMEDA DR,expand exst basement by 550 sq ft entirely wit...,0301020315,...,78735,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MP,Mechanical Permit,2007-105766 MP,Residential,R- 434 Addition & Alterations,Addition,No,1407 ALAMEDA DR,expand exst basement by 550 sq ft entirely wit...,0301020315,...,78735,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MP,Mechanical Permit,2007-107985 MP,Commercial,C-1000 Commercial Remodel,Remodel,No,1524 S IH 35 SVRD SB UNIT 125,interior remodel to existing office Staples ...,0302030941,...,78655,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MP,Mechanical Permit,1982-005467 MP,Residential,R- 101 Single Family Houses,New,No,2104 Wychwood Drive A 00000,2 Stry Frm Res WMas Ven And Att Gar,0105170437,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EP,Electrical Permit,2007-111551 EP,Commercial,"C- 437 Addn, Alter, Convn-NonRes",Addition,No,1201 BARBARA JORDAN BLVD BLDG 13,ADDITION CHIPOTLE MEXICAN GRILL,0221130406,...,78212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In this dataset we will attempt to use the Permit Type as the class label for the data and because of that we need to remove some columns that also act as labels in that capacity that might skew the results. Columns that are similar to Permit type are:


Permit Type Description

Permit Num (since it contain the type in the code)

Permit Class (functions much like a label)

Contractor Trade (plummers typically take plumbing jobs, electricians take electrician jobs thus could act as a label)

Explicit Job indicators: (subject to testing, but due to domain knowledge have opted to remove)

    *Plumbing Valuation
    *Plumbing Valuation Remodel
    *Electrical Valuation
    *Electrical Valuation Remodel
    *Mechanical Valuation
    *Mechanical Valuation Remodel 
    *MedGas Valuation
    *MedGas Valuation Remodel

It might be interesting to note that a renovation job might include plumbing costs and the difference being the cost which decides if its specifically a plumbing job.